In [ ]:
addprocs(11)

In [1]:
import FunctionalCollections
import Iterators
import Pipe
import Compat
import JLD
@everywhere using FunctionalCollections
@everywhere using Iterators
@everywhere using Pipe
@everywhere using Compat
@everywhere using JLD

macro printval(ee)
    ee_expr = @sprintf "%s" string(ee)
    esc(:(println($ee_expr," = ", $ee)))
end

macro pz(ee)
    ee_expr = @sprintf "%s" string(ee)
    esc(:(println($ee_expr,"\t\t",typeof($ee), "\t", size($ee))))
end

push!(LOAD_PATH, ".")
push!(LOAD_PATH, "../util/")

4-element Array{ByteString,1}:
 "/home/ubuntu/build/julia-master/usr/local/share/julia/site/v0.5"
 "/home/ubuntu/build/julia-master/usr/share/julia/site/v0.5"      
 "."                                                              
 "../util/"                                                       

In [ ]:
#shuffled_indexes = 1:length(ground_sents) |> collect |> shuffle!
#nfolds=10
#fold_indexes = Vector{Int}[
#    shuffled_indexes[(ii-1)*end÷nfolds + 1: ii*end÷nfolds]
#    for ii in 1:nfolds]

#@save("brown_glove_folds.jld", fold_indexes)

In [ ]:
@everywhere fold_indexes=load("brown_glove_folds.jld","fold_indexes")

@everywhere function fold_split(fold_ii, raw_bow_res)
    ground_sents = Vector{ASCIIString}[rset[1] for rset in raw_bow_res]
    reconstructed_bows = Vector{ASCIIString}[rset[2] for rset in raw_bow_res]
    
    test_indexes = fold_indexes[fold_ii]
    training_indexes = trues(ground_sents)
    training_indexes[test_indexes]=false

    test_unordered_sents = reconstructed_bows[test_indexes]
    test_ground = ground_sents[test_indexes]
    training_sents = ground_sents[training_indexes]
    test_unordered_sents,test_ground, training_sents
end
    

In [2]:
import PyCall
@everywhere using PyCall
#http://www.nltk.org/howto/probability.html
@everywhere @pyimport nltk
@everywhere @pyimport nltk.probability as nltk_prob

@everywhere function train_language_model{T}(train_corpus::Vector{Vector{T}})
    function py_collect(xs::PyObject)
        xst = []
        for x in xs
            push!(xst,x)
        end
        xst
    end
    function trigram_buffer(sent)
        [START_MARKER1, START_MARKER2, sent..., END_MARKER1, END_MARKER2] 
    end

    training_trigrams = vcat([py_collect(nltk.trigrams(trigram_buffer(sent))) for sent in train_corpus]...)
    kn_prob_dist = nltk_prob.KneserNeyProbDist(pycall(nltk_prob.FreqDist, PyObject, training_trigrams))
    
    function trigram_model(given1::S, given2::S, event::S)
        kn_prob_dist[:prob]((given1, given2, event))
    end
end

/usr/local/lib/python2.7/dist-packages/nltk/app/__init__.py:29: UserWarning: nltk.app package not loaded (please install Tkinter library).
  warnings.warn("nltk.app package not loaded "
/usr/local/lib/python2.7/dist-packages/nltk/draw/__init__.py:15: UserWarning: nltk.draw package not loaded (please install Tkinter library).
  warnings.warn("nltk.draw package not loaded "


In [3]:
@everywhere const START_MARKER1 = "**START1**"
@everywhere const START_MARKER2 = "**START2**"
@everywhere const END_MARKER1 = "**END1**"
@everywhere const END_MARKER2 = "**END2**"


In [4]:
@everywhere typealias S ASCIIString
@everywhere typealias State{T} Tuple{T,T}
@everywhere typealias OrderOptionsCache Dict{Tuple{State{S}, Vector{S}}, Vector{Tuple{plist{S}, Float64}}}


#"""
#returns all possible orderings of the remaining words.
#the freewords are also inserted into every possible position, EXCEPT at the end.
#Freewords also will not be inserted after other freewords
#(For now)
#"""
@everywhere function get_all_orders(unordered_words::Vector{S}, languauge_model::Function; beam_width=Inf)
    _get_options_cache = OrderOptionsCache()

    function transition_prob(cur_state::State{S}, next_word::S)
        languauge_model(cur_state[1],cur_state[2], next_word)
    end
    
    
    function get_options(state::State{S}, remaining_words)
        if length(remaining_words)>0
            get!(_get_options_cache, (state, remaining_words)) do
                _get_options(state, remaining_words)
            end
        else
            _get_options(state)
        end
    end
    
    function _get_options(cur_state::State{S})
        tp = transition_prob(cur_state, END_MARKER1)
        # Given P(END_MARKER2 | curstatep[2]==END_MARKER1) = 1.0
        # Do not need to consider P(END_MARKER2 | curstatep[2]==END_MARKER1, curstatep[1])
        [(EmptyList{S}(), tp)]
    end
    
    function _get_options(cur_state::State{S}, remaining_words)
        function inner()
            (@task begin
                branches = Tuple{Int, Float64}[(ii,transition_prob(cur_state, next_word)) 
                                                for (ii, next_word) in enumerate(remaining_words)]
                beam_end = min(length(remaining_words), beam_width)
                beam = select!(branches, 1:beam_end, by=ip->ip[2], rev=true)
                
                for (next_word_ii, tp) in beam
                    if tp==0.0
                        break #There are No good solutions left (as it is sorted best first)
                    end
                    
                    @inbounds word = remaining_words[next_word_ii]
                    new_remaining_words = sub(remaining_words,[1:next_word_ii-1; next_word_ii+1:length(remaining_words)])
                    @inbounds next_state = (cur_state[2],word)
                    tails_and_tailprobs = get_options(next_state, new_remaining_words) #Actually doing a beam depth-first
                    
                    for (tail, tailprob) in tails_and_tailprobs
                        total_prob = tp*tailprob
                        if total_prob>0.0
                            produce(cons(word, tail), total_prob)
                        end
                    end
                end
            end)
        end
        
        inner() |> collect
    end
    
    initial_state = (START_MARKER1, START_MARKER2)
    get_options(initial_state, unordered_words)
end

In [5]:
function order(unordered_words::Vector{S}, language_model::Function; beam_width=Inf, best_n=1::Int)
    orders_and_probs = get_all_orders(free_words, unordered_words, beam_width)
    best_n = min(best_n, length(orders_and_probs))
    if best_n==0 #None found
        #warn("No possible Orderding found. Defaulting to unordered: ".*string(unordered_words))
        Tuple[(unordered_words, 0.0)]
    elseif best_n==1 #4x as fast as the else
        max_prob, max_ii = @pipe orders_and_probs |> map(op->op[2],_) |> findmax
        Tuple[orders_and_probs[max_ii]]
    else
        select!(orders_and_probs, 1:best_n, by=op->-op[2] )
    end
end

function norm_order(unordered_words::Vector{S}, language_model::Function;  kwargs...)
    orders_and_probs = order(unordered_words, language_model; kwargs...)
    total_prob = @pipe orders_and_probs |> map(op->op[2], _ ) |> sum
    total_prob = total_prob == 0.0 ? 1.0 : total_prob
    [(join(order, " "), prob/total_prob) for (order,prob) in orders_and_probs]
end

@everywhere function best_order(unordered_words::Vector{S}, language_model::Function;  kwargs...)
    orders_and_probs = get_all_orders(unordered_words, language_model; kwargs...)
    if length(orders_and_probs)==0
        return (unordered_words, 0.0)
    end
        
    total_prob = Pipe.@pipe orders_and_probs |> map(op->op[2], _ ) |> sum
    max_prob, max_ii = Pipe.@pipe orders_and_probs |> map(op->op[2],_) |> findmax
    order, prob = orders_and_probs[max_ii]
    @assert(prob==max_prob)
    order, prob/total_prob
end
    

In [6]:
@pyimport nltk.corpus as nltk_corpus
corpus_reader=nltk_corpus.brown
corpus = Vector{ASCIIString}[[lowercase(word) for word in sent] for sent in (corpus_reader[:sents]()|> collect)]

57340-element Array{Array{ASCIIString,1},1}:
 ASCIIString["the","fulton","county","grand","jury","said","friday","an","investigation","of"  …  "``","no","evidence","''","that","any","irregularities","took","place","."]         
 ASCIIString["the","jury","further","said","in","term-end","presentments","that","the","city"  …  "for","the","manner","in","which","the","election","was","conducted","."]           
 ASCIIString["the","september-october","term","jury","had","been","charged","by","fulton","superior"  …  "primary","which","was","won","by","mayor-nominate","ivan","allen","jr.","."]
 ASCIIString["``","only","a","relative","handful","of","such","reports","was","received"  …  "of","voters","and","the","size","of","this","city","''","."]                            
 ASCIIString["the","jury","said","it","did","find","that","many","of","georgia's"  …  "``","are","outmoded","or","inadequate","and","often","ambiguous","''","."]                     
 ASCIIString["it","recommended","that","

In [7]:
lm = train_language_model(corpus)


trigram_model (generic function with 1 method)

In [11]:
lm(START_MARKER1,START_MARKER2, "the")

0.11957185211021974

In [15]:
bow_res=load("results/bags/example200_res.jld", "res")


8-element Array{Tuple{Array{SubString{ASCIIString},1},Array{ASCIIString,1},Float64},1}:
 (SubString{ASCIIString}["name","this","1922","novel","about","leopold","bloom","written","by","james","joyce"],ASCIIString["written","novel","by","this","leopold","joyce","name","bloom","james","about","1922"],-0.0)                                                                                                                                                                                            
 (SubString{ASCIIString}["ralph","waldo","emerson","dismissed","this","poet","as","the","jingle","man"  …  "russell","lowell","called","him","three-fifths","genius","and","two-fifths","sheer","fudge"],ASCIIString["he","him","john","of","james","great","poet","emerson","one","fudge"  …  "949","continued","50-year-old","tappets","naomi","thwart","distally","inaugurated","group","''"],-9.368012441627833)
 (SubString{ASCIIString}["this","is","the","basis","of","a","comedy","of","manners","first","performed

In [17]:
bow_res

8-element Array{Tuple{Array{SubString{ASCIIString},1},Array{ASCIIString,1},Float64},1}:
 (SubString{ASCIIString}["name","this","1922","novel","about","leopold","bloom","written","by","james","joyce"],ASCIIString["written","novel","by","this","leopold","joyce","name","bloom","james","about","1922"],-0.0)                                                                                                                                                                                            
 (SubString{ASCIIString}["ralph","waldo","emerson","dismissed","this","poet","as","the","jingle","man"  …  "russell","lowell","called","him","three-fifths","genius","and","two-fifths","sheer","fudge"],ASCIIString["he","him","john","of","james","great","poet","emerson","one","fudge"  …  "949","continued","50-year-old","tappets","naomi","thwart","distally","inaugurated","group","''"],-9.368012441627833)
 (SubString{ASCIIString}["this","is","the","basis","of","a","comedy","of","manners","first","performed

In [ ]:
map

In [20]:
ordered_res = map(bow_res) do rt
    ground_order, bow, score = rt
    generated_order, prob = best_order(bow, lm; beam_width=5)
    println(generated_order)
    (ground_order, generated_order, prob)
end

ASCIIString["written","novel","by","this","leopold","joyce","name","bloom","james","about","1922"]
ASCIIString["he","him","john","of","james","great","poet","emerson","one","fudge","genius","russell","dismiss","waldo","devoid","jingle","lowell","three-fifths","tops","termed","umber","949","continued","50-year-old","tappets","naomi","thwart","distally","inaugurated","group","''"]
ASCIIString["a","of","in","this","the","comedy","first","of","performed","basis","manners","1892","is"]
ASCIIString["a","in","a","in","the","the","meets","novel","in","meets","who","congo","novel","marlow","third","kurtz","abandons","patna","sailor","and","another"]
ASCIIString["her","she","her","she","it","a","child","when","all","of","leaves","front","end",";","leaps","vronsky","train","aleksei","absent","assisting","draughty","predicts","having","in"]
ASCIIString["we","at","out","the","sun","setting","looked","."]
ASCIIString(i, went, to, the, kitchen, .)
ASCIIString(how, are

8-element Array{Tuple{Array{SubString{ASCIIString},1},Any,Float64},1}:
 (SubString{ASCIIString}["name","this","1922","novel","about","leopold","bloom","written","by","james","joyce"],ASCIIString["written","novel","by","this","leopold","joyce","name","bloom","james","about","1922"],0.0)                                                                                                                                                                              
 (SubString{ASCIIString}["ralph","waldo","emerson","dismissed","this","poet","as","the","jingle","man"  …  "russell","lowell","called","him","three-fifths","genius","and","two-fifths","sheer","fudge"],ASCIIString["he","him","john","of","james","great","poet","emerson","one","fudge"  …  "949","continued","50-year-old","tappets","naomi","thwart","distally","inaugurated","group","''"],0.0)
 (SubString{ASCIIString}["this","is","the","basis","of","a","comedy","of","manners","first","performed","in","1892"],ASCIIString["a","of","in","this"

, you, doing, ?)


In [ ]:

function test(fold_ii)
    test_bows, test_ground_order, train = fold_split(fold_ii,raw_bow_res)
    r_language_models =Dict([pid=>remotecall(pid, train_language_model, train) for pid in workers()])
    pmap([("a","man", "did"),("the","woman", "did")], err_stop=true) do abc
        lm = fetch(r_language_models[myid()])
        lm(abc...)
        
    end
end

test(2)

In [ ]:
#Prepare Oracle Results -- no regen
test_set = load("results/data/brown_glove300.jld")["corpus"]
oracle_res = pmap(test_set, err_stop=true) do target_sent
    score=0.0
    sol = shuffle(target_sent)
    (target_sent, sol, score)
end
@save "results/bags/oracle_res.jld" oracle_res

In [ ]:
@everywhere const len_cap = 18
@everywhere raw_bow_res = load("results/bags/oracle_res.jld", "oracle_res")

function process_fold(fold_ii)
    test_bows, test_ground_order, train = fold_split(fold_ii,raw_bow_res)
    
    #Avoid "serialising a pointer" by getting each process to create their own copy of Language model
    #This also means that the language models are independent (as under the hood they are not readonly, readomg them changes them as they have cache)
    r_language_models =Dict([pid=>remotecall(pid, train_language_model, train) for pid in workers()])
    pmap(test_ground_order, test_bows, err_stop=true) do ground_order, bow
        lm = fetch(r_language_models[myid()])
        generated_order, prob = if length(ground_order)<=len_cap
            best_order(bow, lm; beam_width=5)
        else
            (bow, NaN)
        end
        (ground_order, generated_order, prob)
    end
end

jldopen("results/ordered/oracle_ordered.jld", "w") do file
    for fold_ii in 1:length(fold_indexes)
        res = process_fold(fold_ii)
        write(file, "fold_$(fold_ii)", res)
    end
end


In [ ]:
test_bows, test_ground_order, train = fold_split(2,raw_bow_res)
language_model = train_language_model(train)



In [ ]:
@time ord, prob = best_order(test_bows[4110], language_model; beam_width=5)

In [ ]:
res = pmap(test_bow, err_stop=true) do bow
    order, prob = best_order(bow, language_model; beam_width=5)
    (target_sent, sol, score)
end


In [ ]:
5^20

In [ ]:
@pipe test |> map(x->length(x)==50, _) |> find

In [ ]:
@time best_order(test[60], language_model, beam_width=5)

In [ ]:
ground_sents[16]

In [ ]:
training_trigrams = [nltk.trigrams(sent)|>collect for sent in train]


In [ ]:
map(examples) do unordered_words
    order(unordered_words, ASCIIString[])
    end

In [ ]:
short_cases = Bool[length(ws) <=20 for ws in unordered_output]

true_ordered_sents = test_set[short_cases]
ordered_sents_and_probs = pmap(unordered_output[short_cases]) do unordered_words
    order(unordered_words, zeroed_words)
end

In [ ]:
ordered_sents = map(op->op[1], ordered_sents_and_probs, be)

In [ ]:
perfect_matches = Bool[]  
for ii in 1:length(ordered_sents)
    ordered_words = ordered_sents[ii]
    actual_words = true_ordered_sents[ii]
    
    match = ordered_words == actual_words
    push!(perfect_matches, match)
    #println("$ii - $match")
end
mean(perfect_matches)

In [ ]:
@pyimport nltk
@pyimport nltk.translate.bleu_score as nltk_bleu

function bleu_score(candidate, reference)
    reference = reference |> collect
    candidate = candidate |> collect
    
    if reference==candidate #Perfect Match
        1.0
    else
        weights = [1,1,1,1]/4
        nltk_bleu.bleu(Any[reference],candidate, weights)
    end
end

In [ ]:
map(bleu_score, ordered_sents,true_ordered_sents) |> mean


In [ ]:
bleu_score(true_ordered_sents[3], true_ordered_sents[3])

In [ ]:
true_ordered_sents

In [ ]:
ordered_sents

In [ ]:
ordered_sents[50]

In [ ]:
unordered_output[short_cases][eval_cases][50]

In [ ]:
true_ordered_sents[50]

In [ ]:
#x= ["A", "B", "C", "D"]
#y= UTF8String["A", "B", "C", "D"]
x = true_ordered_sents[10]
y=ordered_sents[10] |> collect
pycall(nltk_bleu.bleu, PyAny, Any[x], y, Any[0.25, 0.25, 0.25, 0.25])

In [ ]:
Any[true_ordered_sents[1]]


In [ ]:
nltk_bleu._modified_precision(Any[reference1, reference2, reference3],candidate1, 1)

In [ ]:
@pyimport pdb

In [ ]:
pdb.runcall(nltk_bleu._modified_precision, [reference1, reference2, reference3],candidate1, 1)

In [ ]:
# I think I have to reimplement BLEU in julia as for some reason it does not play nice with PyCall
# Can basically port http://www.nltk.org/_modules/nltk/align/bleu_score.html#bleu

In [ ]:
pycall(nltk_bleu.bleu, Int, candidate1, [reference1], weights)

In [ ]:
@pyimport nltk.util as nltk_util
nltk_util.ngrams(candidate1,2) |> py_collections.Counter

In [ ]:
@pyimport collections as py_collections

In [ ]:
@pyimport nltk.util as nltk_util
ngs = nltk_util.ngrams(candidate1,2)
pycall(py_collections.Counter, PyObject, ngs)

In [ ]:
import nltk